In [57]:
import keras
from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

keras.__version__

'2.3.1'

In [63]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same',input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding = 'same',activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding = 'same',activation='relu'))

# In the last but one convolutional layer I use filter size which reduces the output to (None,1,1,Num of filters) 
# See model summary to check that the filter size must be (7,7)
model.add(layers.Conv2D(128, (7,7),activation='relu'))

# in the last convolutional layer use the filter size (1,1) and the number of filters equal to the number of classes. Activation should be 'softmax'
model.add(layers.Conv2D(10, (1,1),activation='softmax'))

# The layer output shape is (None,1,1,10) so I flatten it to (None,10) - I reduce redundant internal dimensions 
model.add(layers.Flatten())

In [64]:
model.summary()

# Check the shape of the last layer output - the train_labels must fit this shape exactly!

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 7, 7, 64)          36928     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 1, 1, 128)         401536    
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 1, 1, 10)        

In [65]:

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [66]:

checkpoint = ModelCheckpoint(
    'model.h5', 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True
)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split = 0.2,callbacks = [checkpoint])


Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 41s 861us/step - loss: 0.1529 - accuracy: 0.9519 - val_loss: 0.0735 - val_accuracy: 0.9783

Epoch 00001: val_loss improved from inf to 0.07347, saving model to model.h5
Epoch 2/5
48000/48000 [==============================] - 41s 862us/step - loss: 0.0449 - accuracy: 0.9858 - val_loss: 0.0509 - val_accuracy: 0.9853

Epoch 00002: val_loss improved from 0.07347 to 0.05094, saving model to model.h5
Epoch 3/5
48000/48000 [==============================] - 42s 877us/step - loss: 0.0313 - accuracy: 0.9909 - val_loss: 0.0308 - val_accuracy: 0.9903

Epoch 00003: val_loss improved from 0.05094 to 0.03079, saving model to model.h5
Epoch 4/5
48000/48000 [==============================] - 42s 871us/step - loss: 0.0217 - accuracy: 0.9936 - val_loss: 0.0337 - val_accuracy: 0.9903

Epoch 00004: val_loss did not improve from 0.03079
Epoch 5/5
48000/48000 [==============================] - 42s 870

In [67]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss,test_acc)

predictions = np.argmax(model.predict(test_images),-1)
true_labels = np.argmax(test_labels,-1)

cm = confusion_matrix(true_labels, predictions)
print(cm)
print(classification_report(true_labels, predictions))


10000/10000 [==============================] - 1s 96us/step
0.02480221056835635 0.9923999905586243
[[ 979    0    0    0    0    0    0    1    0    0]
 [   0 1132    1    0    0    0    2    0    0    0]
 [   1    0 1028    0    0    0    0    3    0    0]
 [   0    0    0 1006    0    3    0    0    1    0]
 [   0    0    0    0  972    0    1    1    1    7]
 [   1    0    0    4    0  886    1    0    0    0]
 [   5    2    0    0    1    4  944    0    2    0]
 [   0    1    8    0    0    0    0 1015    1    3]
 [   3    0    2    1    0    1    0    0  964    3]
 [   2    0    1    0    2    4    0    1    1  998]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       1.00      1.00      1.00      1135
           2       0.99      1.00      0.99      1032
           3       1.00      1.00      1.00      1010
           4       1.00      0.99      0.99       982
           5       0.99      0.99      0.99  